In [1]:
import pickle

In [2]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_BookCrossing'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing'

In [3]:
MYMEDIALITEPATH='/home/kirana/MyMediaLite/bin'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pickle
from sklearn import metrics

In [5]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df_side_cold.pkl','rb'))

In [6]:
dftrain.reset_index(inplace=True)
dfvalid.reset_index(inplace=True)

In [7]:
dftrain['rating'].describe()

count    390999.000000
mean          7.602083
std           1.842177
min           1.000000
25%           7.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

In [8]:
df=pd.concat([dftrain,dfvalid],axis=0)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [9]:
df.shape

(433671, 18)

In [10]:
user_to_idx={j:i+1 for i, j in enumerate(df['userId'].unique())}
item_to_idx={j:i+1 for i,j in enumerate(df['itemId'].unique())}
idx_to_user={i+1:j for i, j in enumerate(df['userId'].unique())}
idx_to_item={i+1:j for i,j in enumerate(df['itemId'].unique())}

In [11]:
dftrain['user_idx']=[user_to_idx[i] for i in dftrain['userId']]
dftrain['item_idx']=[item_to_idx[i] for i in dftrain['itemId']]
dfvalid['user_idx']=[user_to_idx[i] for i in dfvalid['userId']]
dfvalid['item_idx']=[item_to_idx[i] for i in dfvalid['itemId']]

In [12]:
dfinp_train=dftrain[['user_idx','item_idx','rating']]
dfinp_test=dfvalid[['user_idx','item_idx','rating']]
dfinp_train.to_csv(f'{PATH}/inp_train_bookcrossing.csv',index=False,header=False,sep=",")
dfinp_test.to_csv(f'{PATH}/inp_test_bookcrossing.csv',index=False,header=False,sep=",")

In [13]:
TRAINFILEPATH=f'{PATH}/inp_train_bookcrossing.csv'
TESTFILEPATH=f'{PATH}/inp_test_bookcrossing.csv'

In [14]:
dfinp_train.head()

,user_idx,item_idx,rating
0,1,1,9
1,2,2,9
2,3,3,5
3,4,4,6
4,5,5,7


In [15]:
dftrain.reset_index(inplace=True)
dfvalid.reset_index(inplace=True)

In [16]:
dftrain['flag_train']=1
dfvalid['flag_train']=0

### Global Average

In [17]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=GlobalAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_globalavg.txt'

In [18]:
!command {cmd}

=loading_time 0.61
memory 33
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
GlobalAverage  training_time 00:00:00.0028360 RMSE 1.858584 MAE 1.508566 CBD 0.2522574 new users: RMSE 1.89347 MAE 1.52003 CBD 0.25596 new items: RMSE 1.92538 MAE 1.57339 CBD 0.25943  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:00:00.0283680
prediction_time 00:00:00.0483240

memory 34
>

In [19]:
predictions_globalavg=pd.read_csv (f'{DATAPATH}/prediction_globalavg.txt',sep='\t',header=None).iloc[:,2]

In [20]:
dfvalid['pred_globalavg']=predictions_globalavg

In [17]:
def score_by_group (dfvalid,predVar,groupVar='cold_start_group',method='global_avg'):
    scores=[]
    mae_scores=[]
    r2_scores=[]
    groups=dfvalid[groupVar].value_counts().index.tolist()
    for group in groups:
        df=dfvalid.loc[dfvalid[groupVar]==group]
        actual=df['rating']
        score=metrics.mean_squared_error(df['rating'],df[predVar])
        mae_score=metrics.mean_absolute_error(df['rating'],df[predVar])
        r2_score=metrics.r2_score(df['rating'],df[predVar])
        scores.append(score)
        mae_scores.append(mae_score)
        r2_scores.append(r2_score)
    return pd.DataFrame({'method':method, 'mse':scores, 'rmse':np.sqrt(scores), 'mae':mae_scores, 'r2_score':r2_scores, 'Cold Start Group':groups})

In [22]:
dfvalid['pred_globalavg']

0        7.602083
1        7.602083
2        7.602083
3        7.602083
4        7.602083
5        7.602083
6        7.602083
7        7.602083
8        7.602083
9        7.602083
10       7.602083
11       7.602083
12       7.602083
13       7.602083
14       7.602083
15       7.602083
16       7.602083
17       7.602083
18       7.602083
19       7.602083
20       7.602083
21       7.602083
22       7.602083
23       7.602083
24       7.602083
25       7.602083
26       7.602083
27       7.602083
28       7.602083
29       7.602083
           ...   
42642    7.602083
42643    7.602083
42644    7.602083
42645    7.602083
42646    7.602083
42647    7.602083
42648    7.602083
42649    7.602083
42650    7.602083
42651    7.602083
42652    7.602083
42653    7.602083
42654    7.602083
42655    7.602083
42656    7.602083
42657    7.602083
42658    7.602083
42659    7.602083
42660    7.602083
42661    7.602083
42662    7.602083
42663    7.602083
42664    7.602083
42665    7.602083
42666    7

In [23]:
result_globalavg=pd.concat([score_by_group(dfvalid,'pred_globalavg'),score_by_group(dfvalid,'pred_globalavg','cold_start_bucket'),\
score_by_group(dfvalid,'pred_globalavg','flag_train')],axis=0)

In [24]:
result_globalavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,global_avg,3.323359,1.823008,1.478961,-0.001588,No-cold-start
1,global_avg,3.649579,1.910387,1.565508,-0.005318,Item-Cold_start
2,global_avg,3.421264,1.849666,1.482640,-0.000132,User-Cold-Start
3,global_avg,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,global_avg,3.323359,1.823008,1.478961,-0.001588,0
1,global_avg,3.622941,1.903402,1.546677,-0.003799,1
0,global_avg,3.454335,1.858584,1.508566,-0.000031,0


In [25]:
dfvalid['flag_train'].value_counts()

0    42672
Name: flag_train, dtype: int64

### User Average

In [26]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_useravg.txt'

In [27]:
!command {cmd}

=loading_time 0.6
memory 33
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
UserAverage  training_time 00:00:00.0140320 RMSE 1.714193 MAE 1.309033 CBD 0.2459723 new users: RMSE 1.89347 MAE 1.52003 CBD 0.25596 new items: RMSE 1.7117 MAE 1.29179 CBD 0.24959  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:00:00.0283350
prediction_time 00:00:00.0461930

memory 35
>

In [28]:
predictions_useravg=pd.read_csv (f'{DATAPATH}/prediction_useravg.txt',sep='\t',header=None).iloc[:,2]

In [29]:
dfvalid['pred_useravg']=predictions_useravg

In [30]:
result_useravg=pd.concat([score_by_group(dfvalid,'pred_useravg','cold_start_group','useravg'),score_by_group(dfvalid,'pred_useravg','cold_start_bucket','useravg'),\
score_by_group(dfvalid,'pred_useravg','flag_train','useravg')],axis=0)
result_useravg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,useravg,2.832725,1.683070,1.278786,0.146278,No-cold-start
1,useravg,2.713754,1.647348,1.226404,0.252465,Item-Cold_start
2,useravg,3.421264,1.849666,1.482640,-0.000132,User-Cold-Start
3,useravg,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,useravg,2.832725,1.683070,1.278786,0.146278,0
1,useravg,3.074568,1.753445,1.347970,0.148137,1
0,useravg,2.938458,1.714193,1.309033,0.149316,0


### Item Average

In [31]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemavg.txt'

In [32]:
!command {cmd}

=loading_time 0.6
memory 33
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
ItemAverage  training_time 00:00:00.0168360 RMSE 1.952499 MAE 1.536878 CBD 0.2618984 new users: RMSE 1.98422 MAE 1.5446 CBD 0.26566 new items: RMSE 1.92538 MAE 1.57339 CBD 0.25943  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:00:00.0311880
prediction_time 00:00:00.0538620

memory 35
>

In [33]:
predictions_itemavg=pd.read_csv (f'{DATAPATH}/prediction_itemavg.txt',sep='\t',header=None).iloc[:,2]

In [34]:
dfvalid['pred_itemavg']=predictions_itemavg

In [35]:
result_itemavg=pd.concat([score_by_group(dfvalid,'pred_itemavg','cold_start_group','itemavg'),score_by_group(dfvalid,'pred_itemavg','cold_start_bucket','itemavg'),\
score_by_group(dfvalid,'pred_itemavg','flag_train','itemavg')],axis=0)
result_itemavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemavg,3.846133,1.961156,1.521362,-0.159141,No-cold-start
1,itemavg,3.649579,1.910387,1.565508,-0.005318,Item-Cold_start
2,itemavg,3.916179,1.978934,1.517203,-0.144809,User-Cold-Start
3,itemavg,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,itemavg,3.846133,1.961156,1.521362,-0.159141,0
1,itemavg,3.768635,1.941297,1.556852,-0.044166,1
0,itemavg,3.812251,1.952499,1.536878,-0.103648,0


## Neighborhood Methods

### Item KNN Cosine

In [ ]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemknn_cosine.txt \
--recommender-options k=1 --recommender-options num_iter=1'

In [ ]:
!command {cmd}

In [ ]:
predictions_itemknn_cosine=pd.read_csv (f'{DATAPATH}/prediction_itemknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_cosine']=predictions_itemknn_cosine

In [ ]:
result_itemknncosine=pd.concat([score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_group','itemknncosine'),score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_bucket','itemknncosine'),\
score_by_group(dfvalid,'pred_itemknn_cosine','flag_train','itemknncosine')],axis=0)
result_itemknncosine

In [ ]:
import pickle
pickle.dump(result_itemknncosine,file=open(f'{DATAPATH}/result_itemknncosine.pkl','wb'))

### Item KNN Pearson

In [18]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemknn_pearson.txt \
--recommender-options k=1 --recommender-options num_iter=1'

In [ ]:
!command {cmd}

=loading_time 0.75
memory 71
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
ItemKNN k=1 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=15 reg_i=10 num_iter=1 

In [ ]:
predictions_itemknn_pearson=pd.read_csv (f'{DATAPATH}/prediction_itemknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_pearson']=predictions_itemknn_pearson

In [ ]:
result_itemknnpearson=pd.concat([score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_group','itemknnpearson'),score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_bucket','itemknnpearson'),\
score_by_group(dfvalid,'pred_itemknn_pearson','flag_train','itemknnpearson')],axis=0)
result_itemknnpearson

In [ ]:
import pickle
pickle.dump(result_itemknnpearson,file=open(f'{DATAPATH}/result_itemknnpearson.pkl','wb'))

### User KNN Pearson

In [18]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_userknn_pearson.txt \
--recommender-options k=3 --recommender-options num_iter=3'

In [19]:
!command {cmd}

=loading_time 0.65
memory 57
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
UserKNN k=3 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=15 reg_i=10 num_iter=3  training_time 02:04:57.2911010 RMSE 1.77937 MAE 1.374146 CBD 0.2503011 new users: RMSE 1.85757 MAE 1.47391 CBD 0.25412 new items: RMSE 1.71441 MAE 1.33203 CBD 0.24849  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:06:25.9384720
prediction_time 00:06:26.0504480

memory 29284
>

In [20]:
predictions_userknn_pearson=pd.read_csv (f'{DATAPATH}/prediction_userknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_pearson']=predictions_userknn_pearson

In [21]:
result_userknnpearson=pd.concat([score_by_group(dfvalid,'pred_userknn_pearson','cold_start_group','userknnpearson'),score_by_group(dfvalid,'pred_userknn_pearson','cold_start_bucket','userknnpearson'),\
score_by_group(dfvalid,'pred_userknn_pearson','flag_train','userknnpearson')],axis=0)
result_userknnpearson

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknnpearson,3.275533,1.809843,1.387251,0.012826,No-cold-start
1,userknnpearson,2.724928,1.650736,1.274866,0.249388,Item-Cold_start
2,userknnpearson,3.231853,1.797736,1.417786,0.055238,User-Cold-Start
3,userknnpearson,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,userknnpearson,3.275533,1.809843,1.387251,0.012826,0
1,userknnpearson,3.025356,1.739355,1.357276,0.161772,1
0,userknnpearson,3.166157,1.779370,1.374146,0.083397,0


In [24]:
import pickle
pickle.dump(result_userknnpearson,file=open(f'{DATAPATH}/result_userknnpearson.pkl','wb'))

### User KNN Cosine

In [52]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_userknn_cosine.txt '

In [53]:
!command {cmd}

=loading_time 0.66
memory 57
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
UserKNN k=80 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=15 reg_i=10 num_iter=10  training_time 02:04:47.1191450 RMSE 1.762762 MAE 1.360549 CBD 0.2491959 new users: RMSE 1.85759 MAE 1.474 CBD 0.25412 new items: RMSE 1.71442 MAE 1.33196 CBD 0.24849  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:06:47.0904830
prediction_time 00:06:49.9614220

memory 29284
>

In [54]:
predictions_userknn_cosine=pd.read_csv (f'{DATAPATH}/prediction_userknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_cosine']=predictions_userknn_cosine

In [55]:
result_userknncosine=pd.concat([score_by_group(dfvalid,'pred_userknn_cosine','cold_start_group','userknncosine'),score_by_group(dfvalid,'pred_userknn_cosine','cold_start_bucket','userknncosine'),\
score_by_group(dfvalid,'pred_userknn_cosine','flag_train','userknncosine')],axis=0)
result_userknncosine

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknncosine,3.170972,1.780722,1.363100,0.044338,No-cold-start
1,userknncosine,2.724954,1.650744,1.274785,0.249380,Item-Cold_start
2,userknncosine,3.231956,1.797764,1.417912,0.055208,User-Cold-Start
3,userknncosine,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,userknncosine,3.170972,1.780722,1.363100,0.044338,0
1,userknncosine,3.025402,1.739368,1.357265,0.161760,1
0,userknncosine,3.107330,1.762762,1.360549,0.100427,0


## Matrix Factorization

### SVD

In [36]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=MatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_svd.txt'

In [37]:
!command {cmd}

=loading_time 0.6
memory 33
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
MatrixFactorization num_factors=10 regularization=0.015 learn_rate=0.01 learn_rate_decay=1 num_iter=30  training_time 00:00:03.5134510 RMSE 1.932287 MAE 1.53733 CBD 0.2582725 new users: RMSE 1.89347 MAE 1.52003 CBD 0.25596 new items: RMSE 1.92538 MAE 1.57339 CBD 0.25943  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:00:00.0183150
prediction_time 00:00:00.0490880

memory 45
>

In [38]:
predictions_svd=pd.read_csv (f'{DATAPATH}/prediction_svd.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svd']=predictions_svd

In [39]:
result_svd=pd.concat([score_by_group(dfvalid,'pred_svd','cold_start_group','svd'),score_by_group(dfvalid,'pred_svd','cold_start_bucket','svd'),\
score_by_group(dfvalid,'pred_svd','flag_train','svd')],axis=0)
result_svd

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svd,3.819795,1.954430,1.530069,-0.151203,No-cold-start
1,svd,3.649579,1.910387,1.565508,-0.005318,Item-Cold_start
2,svd,3.421264,1.849666,1.482640,-0.000132,User-Cold-Start
3,svd,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,svd,3.819795,1.954430,1.530069,-0.151203,0
1,svd,3.622941,1.903402,1.546677,-0.003799,1
0,svd,3.733731,1.932287,1.537330,-0.080916,0


### SVD Plus Plus

In [45]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=SVDPlusPlus \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_svdplusplus.txt '



In [46]:
cmd

'/home/kirana/MyMediaLite/bin/rating_prediction --training-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_BookCrossing/inp_train_bookcrossing.csv --recommender=SVDPlusPlus --test-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_BookCrossing/inp_test_bookcrossing.csv --prediction-file /home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing/prediction_svdplusplus.txt '

In [47]:
!command {cmd}

=loading_time 0.65
memory 33
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
SVDPlusPlus num_factors=10 regularization=0.015 bias_reg=0.33 frequency_regularization=False learn_rate=0.001 bias_learn_rate=0.7 learn_rate_decay=1 num_iter=30  training_time 00:09:46.7908110 RMSE 1.702269 MAE 1.329919 CBD 0.2442075 new users: RMSE 1.86769 MAE 1.48992 CBD 0.25465 new items: RMSE 1.73991 MAE 1.35548 CBD 0.24968  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:00:00.0553140
prediction_time 00:00:00.0482820

memory 77
>

In [48]:
predictions_svdplusplus=pd.read_csv (f'{DATAPATH}/prediction_svdplusplus.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svdplusplus']=predictions_svdplusplus

result_svdplusplus=pd.concat([score_by_group(dfvalid,'pred_svdplusplus','cold_start_group','svdplusplus'),score_by_group(dfvalid,'pred_svdplusplus','cold_start_bucket','svdplusplus'),\
score_by_group(dfvalid,'pred_svdplusplus','flag_train','svdplusplus')],axis=0)
result_svdplusplus

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svdplusplus,2.738175,1.654743,1.290668,0.174774,No-cold-start
1,svdplusplus,2.830975,1.682550,1.303101,0.220176,Item-Cold_start
2,svdplusplus,3.284872,1.812422,1.440296,0.039739,User-Cold-Start
3,svdplusplus,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,svdplusplus,2.738175,1.654743,1.290668,0.174774,0
1,svdplusplus,3.103105,1.761563,1.380447,0.140231,1
0,svdplusplus,2.897721,1.702269,1.329919,0.161109,0


### BiasedMatrixFactorization

In [49]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=BiasedMatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_biasedmatrixfactorization.txt '



In [50]:
!command {cmd}

=loading_time 0.59
memory 33
ratings range: [1, 10]
training data: 72887 users, 173090 items, 390999 ratings, sparsity 99.9969
test data:     17382 users, 31022 items, 42672 ratings, sparsity 99.99209
BiasedMatrixFactorization num_factors=10 bias_reg=0.01 reg_u=0.015 reg_i=0.015 frequency_regularization=False learn_rate=0.01 bias_learn_rate=1 learn_rate_decay=1 num_iter=30 bold_driver=False loss=RMSE max_threads=1 naive_parallelization=False  training_time 00:00:03.7296090 RMSE 1.779405 MAE 1.353109 CBD 0.2486478 new users: RMSE 1.88224 MAE 1.48253 CBD 0.25545 new items: RMSE 1.69331 MAE 1.28547 CBD 0.24776  new users and items: RMSE 1.99718 MAE 1.61202 CBD 0.26377 testing_time 00:00:00.0205560
prediction_time 00:00:00.0491950

memory 46
>

In [51]:
predictions_biasedmatrixfactorization=pd.read_csv (f'{DATAPATH}/prediction_biasedmatrixfactorization.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_biasedmatrixfactorization']=predictions_biasedmatrixfactorization

result_biasedmatrixfactorization=pd.concat([score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_group','biasedmatrixfactorization'),score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_bucket','biasedmatrixfactorization'),\
score_by_group(dfvalid,'pred_biasedmatrixfactorization','flag_train','biasedmatrixfactorization')],axis=0)
result_biasedmatrixfactorization

,method,mse,rmse,mae,r2_score,Cold Start Group
0,biasedmatrixfactorization,3.285500,1.812595,1.372624,0.009822,No-cold-start
1,biasedmatrixfactorization,2.638329,1.624293,1.218794,0.273242,Item-Cold_start
2,biasedmatrixfactorization,3.361617,1.833471,1.429907,0.017305,User-Cold-Start
3,biasedmatrixfactorization,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,biasedmatrixfactorization,3.285500,1.812595,1.372624,0.009822,0
1,biasedmatrixfactorization,3.012811,1.735745,1.327987,0.165248,1
0,biasedmatrixfactorization,3.166282,1.779405,1.353109,0.083361,0


In [78]:
results_concat=pd.concat ([result_biasedmatrixfactorization,
result_svdplusplus,
result_svd,
result_userknncosine,
result_itemavg,
result_useravg,
result_globalavg],axis=0)

In [79]:
pickle.dump(results_concat,open(f'{DATAPATH}/diff_results.pkl','wb'))

In [80]:
results_concat

,method,mse,rmse,mae,r2_score,Cold Start Group
0,biasedmatrixfactorization,3.285500,1.812595,1.372624,0.009822,No-cold-start
1,biasedmatrixfactorization,2.638329,1.624293,1.218794,0.273242,Item-Cold_start
2,biasedmatrixfactorization,3.361617,1.833471,1.429907,0.017305,User-Cold-Start
3,biasedmatrixfactorization,3.988708,1.997175,1.612015,-0.015393,User-Item-cold-start
0,biasedmatrixfactorization,3.285500,1.812595,1.372624,0.009822,0
1,biasedmatrixfactorization,3.012811,1.735745,1.327987,0.165248,1
0,biasedmatrixfactorization,3.166282,1.779405,1.353109,0.083361,0
0,svdplusplus,2.738175,1.654743,1.290668,0.174774,No-cold-start
1,svdplusplus,2.830975,1.682550,1.303101,0.220176,Item-Cold_start
2,svdplusplus,3.284872,1.812422,1.440296,0.039739,User-Cold-Start


In [81]:
pickle.dump([dftrain,dfvalid,results_concat],open(f'{DATAPATH}/df_side_cold_runother.pkl','wb'))

In [ ]:
result_userknnpearson,
result_itemknnpearson,
result_itemknncosine,